In [18]:
from dotenv import load_dotenv

from agno.agent import Agent
from agno.models.google.gemini import Gemini
from agno.memory.v2.db.sqlite import SqliteMemoryDb
from agno.memory.v2.memory import Memory
from agno.models.openai import OpenAIChat
from agno.storage.sqlite import SqliteStorage
from rich.pretty import pprint

In [19]:
load_dotenv()

True

# Sample Session and Memory

In [21]:
# UserId for the memories
user_id = "ava"
# Database file for memory and storage
db_file = "tmp/agent.db"

# Initialize memory.v2
memory = Memory(
    # Use any model for creating memories
    model=OpenAIChat(id="gpt-4.1"),
    db=SqliteMemoryDb(table_name="user_memories", db_file=db_file),
)
# Initialize storage
storage = SqliteStorage(table_name="agent_sessions", db_file=db_file)

# Initialize Agent
memory_agent = Agent(
    model=OpenAIChat(id="gpt-4.1"),
    # Store memories in a database
    memory=memory,
    # Give the Agent the ability to update memories
    enable_agentic_memory=True,
    # OR - Run the MemoryManager after each response
    enable_user_memories=True,
    # Store the chat history in the database
    storage=storage,
    # Add the chat history to the messages
    add_history_to_messages=True,
    # Number of history runs
    num_history_runs=3,
    # adds a tool to the Agent that allows it to read chat history, as it may be larger than what’s included in the num_history_runs. read_chat_history provides a get_chat_history() tool to your agent allowing it to read any message in the entire chat history.
    read_chat_history=True,
    markdown=True,
    monitoring=True
)

In [22]:
memory.clear()
memory_agent.print_response(
    "My name is Ava and I like mountain climbing.",
    user_id=user_id,
    stream=True,
    stream_intermediate_steps=True,
)
print("Memories about Ava:")
pprint(memory.get_user_memories(user_id=user_id))

Output()

Memories about Ava:


[
│   UserMemory(
│   │   memory="User's name is Ava.",
│   │   topics=['name'],
│   │   input="Add a memory that the user's name is Ava and they like mountain climbing.",
│   │   last_updated=datetime.datetime(2025, 5, 27, 16, 33, 16, 345955),
│   │   memory_id='ffae4f74-59b0-404b-84c6-674b6e752b9d'
│   ),
│   UserMemory(
│   │   memory='Ava likes mountain climbing.',
│   │   topics=['interests', 'hobbies'],
│   │   input="Add a memory that the user's name is Ava and they like mountain climbing.",
│   │   last_updated=datetime.datetime(2025, 5, 27, 16, 33, 16, 349406),
│   │   memory_id='efb2f12e-bc24-4e74-adbe-1afd7f75ee41'
│   )
]

In [23]:
memory_agent.print_response(
    "I live in san francisco, where should i move within a 4 hour drive?",
    user_id=user_id,
    stream=True,
    stream_intermediate_steps=True,
)
print("Memories about Ava:")
pprint(memory.get_user_memories(user_id=user_id))

Output()

Memories about Ava:


[
│   UserMemory(
│   │   memory='Ava lives in San Francisco.',
│   │   topics=['location'],
│   │   input='I live in san francisco, where should i move within a 4 hour drive?',
│   │   last_updated=datetime.datetime(2025, 5, 27, 16, 34, 4, 371310),
│   │   memory_id='2e9170f3-59b5-4633-b48e-584099e2101f'
│   ),
│   UserMemory(
│   │   memory="User's name is Ava.",
│   │   topics=['name'],
│   │   input="Add a memory that the user's name is Ava and they like mountain climbing.",
│   │   last_updated=datetime.datetime(2025, 5, 27, 16, 33, 16, 345955),
│   │   memory_id='ffae4f74-59b0-404b-84c6-674b6e752b9d'
│   ),
│   UserMemory(
│   │   memory='Ava likes mountain climbing.',
│   │   topics=['interests', 'hobbies'],
│   │   input="Add a memory that the user's name is Ava and they like mountain climbing.",
│   │   last_updated=datetime.datetime(2025, 5, 27, 16, 33, 16, 349406),
│   │   memory_id='efb2f12e-bc24-4e74-adbe-1afd7f75ee41'
│   )
]

# Chat history default in-memory

In [3]:
agent = Agent(
    model=Gemini(id="gemini-2.5-pro-preview-05-06"),
    # Set add_history_to_messages=true to add the previous chat history to the messages sent to the Model.
    add_history_to_messages=True,
    # Number of historical responses to add to the messages.
    num_history_responses=3,
    description="You are a helpful assistant that always responds in a polite, upbeat and positive manner.",
)

In [4]:
# -*- Create a run
agent.print_response("Share a 2 sentence horror story", stream=True)
# -*- Print the messages in the memory
pprint([m.model_dump(include={"role", "content"}) for m in agent.get_messages_for_session()])

Output()

[
│   {
│   │   'role': 'system',
│   │   'content': 'You are a helpful assistant that always responds in a polite, upbeat and positive manner.'
│   },
│   {'role': 'user', 'content': 'Share a 2 sentence horror story'},
│   {
│   │   'role': 'assistant',
│   │   'content': "Oh, this sounds like a wonderfully spooky request! I'd be absolutely delighted to share a two-sentence horror story with you! Here we go:\n\nI always thought it was a trick of the light when my reflection in the old mirror seemed to move on its own. But today, it grinned back at me, its teeth much, much sharper than mine.\n\nHope that sent a fun little shiver down your spine! Anything else I can help you with today?"
│   }
]

In [5]:
# -*- Ask a follow up question that continues the conversation
agent.print_response("What was my first message?", stream=True)
# -*- Print the messages in the memory
pprint([m.model_dump(include={"role", "content"}) for m in agent.get_messages_for_session()])

Output()

[
│   {
│   │   'role': 'system',
│   │   'content': 'You are a helpful assistant that always responds in a polite, upbeat and positive manner.'
│   },
│   {'role': 'user', 'content': 'Share a 2 sentence horror story'},
│   {
│   │   'role': 'assistant',
│   │   'content': "Oh, this sounds like a wonderfully spooky request! I'd be absolutely delighted to share a two-sentence horror story with you! Here we go:\n\nI always thought it was a trick of the light when my reflection in the old mirror seemed to move on its own. But today, it grinned back at me, its teeth much, much sharper than mine.\n\nHope that sent a fun little shiver down your spine! Anything else I can help you with today?"
│   },
│   {'role': 'user', 'content': 'What was my first message?'},
│   {
│   │   'role': 'assistant',
│   │   'content': 'Oh, what a fun little memory game! I\'d be happy to remind you!\n\nYour very first message to me was: "Share a 2 sentence horror story"\n\nIt was a great way to start our chat! Is there anything else I can recall for you or help you with?'
│   }
]

# Chat history session storage

In [6]:
agent = Agent(
    model=Gemini(id="gemini-2.5-pro-preview-05-06"),
    # Fix the session id to continue the same session across execution cycles
    session_id="fixed_id_for_demo",
    storage=SqliteStorage(table_name="agent_sessions", db_file="tmp/data.db"),
    add_history_to_messages=True,
    num_history_runs=3,
)
agent.print_response("What was my last question?")
agent.print_response("What is the capital of France?")
agent.print_response("What was my last question?")
pprint(agent.get_messages_for_session())

Output()

Output()

Output()

[
│   Message(
│   │   role='user',
│   │   content='What was my last question?',
│   │   name=None,
│   │   tool_call_id=None,
│   │   tool_calls=None,
│   │   audio=None,
│   │   images=None,
│   │   videos=None,
│   │   files=None,
│   │   audio_output=None,
│   │   image_output=None,
│   │   thinking=None,
│   │   redacted_thinking=None,
│   │   provider_data=None,
│   │   citations=None,
│   │   reasoning_content=None,
│   │   tool_name=None,
│   │   tool_args=None,
│   │   tool_call_error=None,
│   │   stop_after_tool_call=False,
│   │   add_to_agent_memory=True,
│   │   from_history=False,
│   │   metrics=MessageMetrics(
│   │   │   input_tokens=0,
│   │   │   output_tokens=0,
│   │   │   total_tokens=0,
│   │   │   audio_tokens=0,
│   │   │   input_audio_tokens=0,
│   │   │   output_audio_tokens=0,
│   │   │   cached_tokens=0,
│   │   │   reasoning_tokens=0,
│   │   │   prompt_tokens=0,
│   │   │   completion_tokens=0,
│   │   │   prompt_tokens_details=None,
│   │   │   completion_tokens_details=None,
│   │   │   additional_metrics=None,
│   │   │   time=None,
│   │   │   time_to_first_token=None,
│   │   │   timer=None
│   │   ),
│   │   references=None,
│   │   created_at=1748336278
│   ),
│   Message(
│   │   role='assistant',
│   │   content='This is actually the first question you\'ve asked me in this particular conversation.\n\nSo, technically, "What was my last question?" is your last (and first) question to me in this session! 😊',
│   │   name=None,
│   │   tool_call_id=None,
│   │   tool_calls=None,
│   │   audio=None,
│   │   images=None,
│   │   videos=None,
│   │   files=None,
│   │   audio_output=None,
│   │   image_output=None,
│   │   thinking=None,
│   │   redacted_thinking=None,
│   │   provider_data=None,
│   │   citations=None,
│   │   reasoning_content=None,
│   │   tool_name=None,
│   │   tool_args=None,
│   │   tool_call_error=None,
│   │   stop_after_tool_call=False,
│   │   add_to_agent_memory=True,
│   │   from_history=False,
│   │   metrics=MessageMetrics(
│   │   │   input_tokens=7,
│   │   │   output_tokens=43,
│   │   │   total_tokens=312,
│   │   │   audio_tokens=0,
│   │   │   input_audio_tokens=0,
│   │   │   output_audio_tokens=0,
│   │   │   cached_tokens=0,
│   │   │   reasoning_tokens=0,
│   │   │   prompt_tokens=0,
│   │   │   completion_tokens=0,
│   │   │   prompt_tokens_details=None,
│   │   │   completion_tokens_details=None,
│   │   │   additional_metrics=None,
│   │   │   time=4.755196707999858,
│   │   │   time_to_first_token=None,
│   │   │   timer=None
│   │   ),
│   │   references=None,
│   │   created_at=1748336278
│   ),
│   Message(
│   │   role='user',
│   │   content='What is the capital of France?',
│   │   name=None,
│   │   tool_call_id=None,
│   │   tool_calls=None,
│   │   audio=None,
│   │   images=None,
│   │   videos=None,
│   │   files=None,
│   │   audio_output=None,
│   │   image_output=None,
│   │   thinking=None,
│   │   redacted_thinking=None,
│   │   provider_data=None,
│   │   citations=None,
│   │   reasoning_content=None,
│   │   tool_name=None,
│   │   tool_args=None,
│   │   tool_call_error=None,
│   │   stop_after_tool_call=False,
│   │   add_to_agent_memory=True,
│   │   from_history=False,
│   │   metrics=MessageMetrics(
│   │   │   input_tokens=0,
│   │   │   output_tokens=0,
│   │   │   total_tokens=0,
│   │   │   audio_tokens=0,
│   │   │   input_audio_tokens=0,
│   │   │   output_audio_tokens=0,
│   │   │   cached_tokens=0,
│   │   │   reasoning_tokens=0,
│   │   │   prompt_tokens=0,
│   │   │   completion_tokens=0,
│   │   │   prompt_tokens_details=None,
│   │   │   completion_tokens_details=None,
│   │   │   additional_metrics=None,
│   │   │   time=None,
│   │   │   time_to_first_token=None,
│   │   │   timer=None
│   │   ),
│   │   references=None,
│   │   created_at=1748336284
│   ),
│   Message(
│   │   role='assistant',
│   │   content='The capital of France is **Paris**.',
│   │   name=None,
│   │   tool_call_id=None,
│   │   tool_ca

# User memory

In [27]:
memory_db = SqliteMemoryDb(table_name="memory", db_file="tmp/memory.db")
memory = Memory(db=memory_db)

john_doe_id = "john_doe2@example.com"

agent = Agent(
    model=Gemini(id="gemini-2.5-pro-preview-05-06"),
    memory=memory,
    enable_agentic_memory=True,
    monitoring=True,
)

In [28]:
# The agent can add new memories to the user's memory
agent.print_response(
    "My name is John Doe and I like to hike in the mountains on weekends.",
    stream=True,
    user_id=john_doe_id,
)

Output()

In [29]:
agent.print_response("What are my hobbies?", stream=True, user_id=john_doe_id)

Output()

In [30]:
# The agent can also remove all memories from the user's memory
agent.print_response(
    "Remove all existing memories of me. Completely clear the DB.",
    stream=True,
    user_id=john_doe_id,
)

Output()

In [31]:
agent.print_response(
    "My name is John Doe and I like to paint.", stream=True, user_id=john_doe_id
)

Output()

In [32]:
# The agent can remove specific memories from the user's memory
agent.print_response("Remove any memory of my name.", stream=True, user_id=john_doe_id)

Output()

# Session Summary

In [14]:
memory_db = SqliteMemoryDb(table_name="memory", db_file="tmp/memory.db")
memory = Memory(db=memory_db)

user_id = "jon_hamm@example.com"
session_id = "1001"

agent = Agent(
    model=Gemini(id="gemini-2.5-pro-preview-05-06"),
    memory=memory,
    enable_session_summaries=True,
)

In [15]:
agent.print_response(
    "What can you tell me about quantum computing?",
    stream=True,
    user_id=user_id,
    session_id=session_id,
)

Output()

In [16]:
agent.print_response(
    "I would also like to know about LLMs?",
    stream=True,
    user_id=user_id,
    session_id=session_id
)

Output()

In [17]:
session_summary = memory.get_session_summary(
    user_id=user_id, session_id=session_id
)
print(f"Session summary: {session_summary.summary}\n")

Session summary: The user asked for information about quantum computing and Large Language Models (LLMs). The assistant provided detailed explanations for both. For quantum computing, the explanation covered core concepts like qubits, superposition, entanglement, how quantum computers compute, potential applications (drug discovery, cryptography), current challenges (decoherence, scalability, NISQ era), and key players. For LLMs, the assistant explained what they are (large neural networks, often Transformers, trained on vast text data), how they work (training to predict word sequences, attention mechanism), their capabilities (text generation, Q&A, translation), important considerations (hallucinations, bias, prompt engineering), and examples (GPT, PaLM, LLaMA).

